# Databento Python client quickstart

**Welcome to the Databento client library quickstart tutorial!**

We'll walk through how to use our client library's functionality to work with the data available from Databento.

**Note:**

For information on our symbology, refer to https://docs.databento.com/reference-historical/basics/symbology. 

For a more detailed API reference, refer to https://docs.databento.com/reference-historical.

This tutorial covers the following:
- Using the historical client to request metadata
- Using the historical client to request time series market data
- Working with Bento data I/O helper objects
- Using the historical client to make batch data requests
- Querying batch job states

**Tip:** You can call help() on any class or method to see the 'docstring'.

## Installation and setup

Firtly, ensure you have the latest `databento` client library installed:
```bash
pip install -U databento
```

## Historical data client

Once you've installed the Python client library, you can import it and initialize a historical client for requests. We'll use this `client` throughout the rest of the tutorial.

To initialize a client, you need to provide a valid API key. You can find these on the API Keys page of your Databento portal at https://databento.com.

In [1]:
import databento as db

client = db.Historical(key="YOUR_API_KEY")

## Requesting metadata

Before we make any requests for actual data, we can look into the metadata to see what is available to download.

In [4]:
client.metadata.list_datasets()

['GLBX.MDP3', 'XNAS.ITCH']

In [5]:
client.metadata.list_schemas(dataset="GLBX.MDP3")

['mbo',
 'mbp-1',
 'mbp-10',
 'tbbo',
 'trades',
 'ohlcv-1s',
 'ohlcv-1m',
 'ohlcv-1h',
 'ohlcv-1d']

In [6]:
client.metadata.list_fields(dataset="GLBX.MDP3", schema="trades", encoding="csv")

{'GLBX.MDP3': {'csv': {'trades': {'ts_recv': 'int',
    'ts_event': 'int',
    'ts_in_delta': 'int',
    'publisher_id': 'int',
    'product_id': 'int',
    'action': 'string',
    'side': 'string',
    'flags': 'int',
    'price': 'int',
    'size': 'int',
    'sequence': 'int'}}}}

In [7]:
client.metadata.list_encodings()

['dbn', 'csv', 'json']

In [8]:
client.metadata.list_compressions()

['none', 'zstd']

### Unit cost / GB

To get the unit cost / GB of all the different data schemas, use this API call to the `list_unit_prices` function.

In [9]:
client.metadata.list_unit_prices(dataset="GLBX.MDP3", mode="historical-streaming")

{'historical-streaming': {'mbo': 1.1,
  'mbp-1': 2.42,
  'mbp-10': 0.45,
  'tbbo': 17.89,
  'trades': 24.8,
  'ohlcv-1s': 50.5,
  'ohlcv-1m': 63.5,
  'ohlcv-1h': 130.0,
  'ohlcv-1d': 175.0,
  'definition': 1.66,
  'statistics': 0.71,
  'status': 1300.0}}

### Parameter setup for data cost query

First, instantiate a dictionary of the query parameters for the data you're interested in.

In [47]:
params = {
    "dataset": "GLBX.MDP3",
    "symbols": "ES.FUT",
    "stype_in": "smart",
    "schema": "mbo",
    "start": "2022-06-10T14:30",
    "end": "2022-06-11",
}

### Data cost
Before making a request for data, you can query the expected price in USD. The price is determined from the following formula: `unit_cost` * `uncompressed_size_GB`.

In [48]:
client.metadata.get_cost(**params)

0.7184975445270538

## Requesting time series data

The rest of this tutorial involves historical time series data. Here's how to request for this data.

The historical time series data is streamed into an in-memory buffer encapsulated by a `Bento` object, which we'll use later to work with the data.

The following code sample requests for all E-mini S&P500 Futures contracts, active between 2022-06-10T14:30 and 2022-06-11 using `smart` symbology.

In [49]:
data = client.timeseries.stream(
    **params,
    limit=1000,  # <-- request limited to 1000 records
)

## Working with the Bento helper object

All time series data requests include a metadata header with the following specifications:
- The original query paramaters (these can be used to re-request the data)
- Symbology mappings

### Metadata properties

In [50]:
data.dataset

'GLBX.MDP3'

In [51]:
data.schema

<Schema.MBO: 'mbo'>

In [52]:
data.symbols

['ES.FUT']

In [53]:
data.stype_in

<SType.SMART: 'smart'>

In [54]:
data.stype_out

<SType.PRODUCT_ID: 'product_id'>

In [55]:
data.start

Timestamp('2022-06-10 14:30:00+0000', tz='UTC')

In [56]:
data.end

Timestamp('2022-06-11 00:00:00+0000', tz='UTC')

In [57]:
data.limit

1000

In [58]:
data.compression

<Compression.ZSTD: 'zstd'>

In [59]:
data.record_count

1000

In [60]:
data.dtype

dtype([('length', 'u1'), ('rtype', 'u1'), ('publisher_id', '<u2'), ('product_id', '<u4'), ('ts_event', '<u8'), ('order_id', '<u8'), ('price', '<i8'), ('size', '<u4'), ('flags', 'i1'), ('channel_id', 'u1'), ('action', 'S1'), ('side', 'S1'), ('ts_recv', '<u8'), ('ts_in_delta', '<i4'), ('sequence', '<u4')])

In [61]:
data.struct_size

56

### Symbology resolution

The metadata contains all information which would have been provided in a `symbology.resolve` request:

In [62]:
data.symbology

{'symbols': ['ES.FUT'],
 'stype_in': 'smart',
 'stype_out': 'product_id',
 'start_date': '2022-06-10',
 'end_date': '2022-06-11',
 'partial': [],
 'not_found': [],
 'mappings': {'ESH3': [{'start_date': datetime.date(2022, 6, 10),
    'end_date': datetime.date(2022, 6, 11),
    'symbol': '206299'}],
  'ESH3-ESZ3': [{'start_date': datetime.date(2022, 6, 10),
    'end_date': datetime.date(2022, 6, 11),
    'symbol': '16567'}],
  'ESM2-ESH3': [{'start_date': datetime.date(2022, 6, 10),
    'end_date': datetime.date(2022, 6, 11),
    'symbol': '6817'}],
  'ESM2-ESM3': [{'start_date': datetime.date(2022, 6, 10),
    'end_date': datetime.date(2022, 6, 11),
    'symbol': '25719'}],
  'ESM3-ESU3': [{'start_date': datetime.date(2022, 6, 10),
    'end_date': datetime.date(2022, 6, 11),
    'symbol': '28177'}],
  'ESM4-ESU4': [{'start_date': datetime.date(2022, 6, 10),
    'end_date': datetime.date(2022, 6, 11),
    'symbol': '4155'}],
  'ESU2-ESH3': [{'start_date': datetime.date(2022, 6, 10),
   

### Symbology mappings

A subset of the symbology metadata includes mappings — per date interval — between the requested symbols `stype_in` and the specified `stype_out`.

In [63]:
data.mappings

{'ESH3': [{'start_date': datetime.date(2022, 6, 10),
   'end_date': datetime.date(2022, 6, 11),
   'symbol': '206299'}],
 'ESH3-ESZ3': [{'start_date': datetime.date(2022, 6, 10),
   'end_date': datetime.date(2022, 6, 11),
   'symbol': '16567'}],
 'ESM2-ESH3': [{'start_date': datetime.date(2022, 6, 10),
   'end_date': datetime.date(2022, 6, 11),
   'symbol': '6817'}],
 'ESM2-ESM3': [{'start_date': datetime.date(2022, 6, 10),
   'end_date': datetime.date(2022, 6, 11),
   'symbol': '25719'}],
 'ESM3-ESU3': [{'start_date': datetime.date(2022, 6, 10),
   'end_date': datetime.date(2022, 6, 11),
   'symbol': '28177'}],
 'ESM4-ESU4': [{'start_date': datetime.date(2022, 6, 10),
   'end_date': datetime.date(2022, 6, 11),
   'symbol': '4155'}],
 'ESU2-ESH3': [{'start_date': datetime.date(2022, 6, 10),
   'end_date': datetime.date(2022, 6, 11),
   'symbol': '431108'}],
 'ESU2-ESU3': [{'start_date': datetime.date(2022, 6, 10),
   'end_date': datetime.date(2022, 6, 11),
   'symbol': '16445'}],
 'ESU

### Instrument definitions

The metadata also contains 'mini-definitions,' which are a subset of the full `definition` schema. The full instrument definitions — including all data from the exchange — can be obtained in a separate request.

### Pandas DataFrame

To construct a pandas `DataFrame` from the data, you can call the `.to_df()` method.

In [64]:
import pandas as pd

pd.set_option('display.max_columns', None)

df = data.to_df(pretty_px=True, pretty_ts=True)
df.head(20)

,ts_event,ts_in_delta,publisher_id,channel_id,product_id,order_id,action,side,flags,price,size,sequence
ts_recv,,,,,,,,,,,,
2022-06-10 14:30:00.000637392+00:00,2022-06-10 14:30:00.000025147+00:00,16157,1,0,97042,6410153029859,A,B,128,3903.50,10,65509273
2022-06-10 14:30:00.000758204+00:00,2022-06-10 14:30:00.000644983+00:00,18570,1,0,97042,6410153029860,A,B,128,3903.50,1,65509274
2022-06-10 14:30:00.003570400+00:00,2022-06-10 14:30:00.003445373+00:00,21408,1,0,3403,6410153029861,A,A,128,3904.75,1,65509275
2022-06-10 14:30:00.003759884+00:00,2022-06-10 14:30:00.003627981+00:00,20756,1,0,3403,6410153029862,A,B,128,3895.00,1,65509276
2022-06-10 14:30:00.003766076+00:00,2022-06-10 14:30:00.003630375+00:00,16130,1,0,3403,6410153029863,A,A,128,3904.50,1,65509277
2022-06-10 14:30:00.003773116+00:00,2022-06-10 14:30:00.003651355+00:00,17362,1,0,3403,6410153029864,A,A,128,3904.75,1,65509278
2022-06-10 14:30:00.003786908+00:00,2022-06-10 14:30:00.003657773+00:00,25327,1,0,3403,6410153029865,A,B,128,3894.75,1,65509279
2022-06-10 14:30:00.003787568+00:00,2022-06-10 14:30:00.003675667+00:00,15046,1,0,3403,6410153029866,A,B,128,3894.50,1,65509280
2022-06-10 14:30:00.003802408+00:00,2022-06-10 14:30:00.003690029+00:00,20556,1,0,3403,6410153029867,A,A,128,3905.00,1,65509281


### Numpy arrays

To cast the data to an array of individual records represented as `np.ndarray`(s), you can call the `to_ndarray()` method.



In [65]:
data.to_ndarray()[:10]

array([(14, 160, 1, 97042, 1654871400000025147, 6410153029859, 3903500000000, 10, -128, 0, b'A', b'B', 1654871400000637392, 16157, 65509273),
       (14, 160, 1, 97042, 1654871400000644983, 6410153029860, 3903500000000,  1, -128, 0, b'A', b'B', 1654871400000758204, 18570, 65509274),
       (14, 160, 1,  3403, 1654871400003445373, 6410153029861, 3904750000000,  1, -128, 0, b'A', b'A', 1654871400003570400, 21408, 65509275),
       (14, 160, 1,  3403, 1654871400003627981, 6410153029862, 3895000000000,  1, -128, 0, b'A', b'B', 1654871400003759884, 20756, 65509276),
       (14, 160, 1,  3403, 1654871400003630375, 6410153029863, 3904500000000,  1, -128, 0, b'A', b'A', 1654871400003766076, 16130, 65509277),
       (14, 160, 1,  3403, 1654871400003651355, 6410153029864, 3904750000000,  1, -128, 0, b'A', b'A', 1654871400003773116, 17362, 65509278),
       (14, 160, 1,  3403, 1654871400003657773, 6410153029865, 3894750000000,  1, -128, 0, b'A', b'B', 1654871400003786908, 25327, 65509279),
      

### Replay

To replay the time series data stream record by record to a handlers callback, you can use the `.replay(callback)` method.

In [32]:
def my_handler(record):
    # backtesting / trading strategies (event-driven)
    print(record)

In [ ]:
#  data.replay(my_handler)

## Writing data to disk

You can write the raw DBN data to disk using the `.to_file()` method.



In [33]:
data.to_file("test.dbn")

You can also write to disk as CSV or JSON.

In [34]:
data.to_csv("my_data.csv")

In [35]:
data.to_json("my_data.json")

## Time series batch requests

The client library can also make batch data requests to the Databento API.

In [2]:
client.batch.submit_job(
    dataset="GLBX.MDP3",
    symbols=["ESH1"],
    schema="trades",
    start="2020-12-27T12:00",
    end="2020-12-29",
    encoding="dbn",
    delivery="download",
    limit=1000,  # <-- request limited to 1000 records
)

[('dataset', 'glbx.mdp3'), ('start', '2020-12-27T12:00:00'), ('end', '2020-12-29T00:00:00'), ('symbols', 'ESH1'), ('schema', 'trades'), ('stype_in', 'native'), ('stype_out', 'product_id'), ('limit', '1000'), ('encoding', 'dbn'), ('split_duration', 'day'), ('packaging', 'none'), ('delivery', 'download')]


{'id': 'GLBX-20221106-EUV99KMLFQ',
 'user_id': '46PCMCVF',
 'bill_id': '57b69b21d9154fc3b047d740d548555d',
 'cost': 0.110865,
 'dataset': 'GLBX.MDP3',
 'symbols': 'ESH1',
 'stype_in': 'native',
 'stype_out': 'product_id',
 'schema': 'trades',
 'start': '2020-12-27 12:00:00+00:00',
 'end': '2020-12-29 00:00:00+00:00',
 'limit': 1000,
 'encoding': 'dbn',
 'compression': 'zstd',
 'pretty_px': False,
 'pretty_ts': False,
 'split_duration': 'day',
 'split_size': None,
 'split_symbols': False,
 'packaging': 'none',
 'delivery': 'download',
 'is_full_universe': False,
 'is_example': False,
 'record_count': 1000,
 'billed_size': 48000,
 'actual_size': None,
 'package_size': None,
 'state': 'queued',
 'ts_received': '2022-11-06 21:38:51.679646+00:00',
 'ts_queued': None,
 'ts_process_start': None,
 'ts_process_done': None,
 'ts_expiration': None}

## Querying batch job state

It's possible to query for a list of your batch jobs, with optional filter parameters for `state` and `since`. This could help to programatically build and manage larger data pipelines.

In [ ]:
client.batch.list_jobs(since=pd.Timestamp.utcnow() - pd.Timedelta(minutes=5))

[{'id': 'GLBX-20220720-BTW9J5HY5C',
  'user_id': '46PCMCVF',
  'bill_id': '3eaf1158',
  'dataset': 'GLBX.MDP3',
  'symbols': 'ESH1',
  'stype_in': 'native',
  'stype_out': 'product_id',
  'schema': 'trades',
  'start': '2020-12-27 12:00:00+00:00',
  'end': '2020-12-29 00:00:00+00:00',
  'limit': 100,
  'encoding': 'dbn',
  'compression': 'zstd',
  'split_duration': 'day',
  'split_size': None,
  'packaging': 'none',
  'delivery': 'download',
  'is_example': False,
  'record_count': 100,
  'billed_size': 4800,
  'actual_size': None,
  'package_size': None,
  'state': 'queued',
  'ts_received': '2022-07-20 07:26:45.617296+00:00',
  'ts_queued': '2022-07-20 07:26:46.395321+00:00',
  'ts_process_start': None,
  'ts_process_done': None,
  'ts_expiration': None,
  'progress': 0}]